In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('C:\\Users\\abner\\proyecto\\Proyecto-\\DB\\datos_limpios.csv')
df.head(3)

,id_extraccion,timestamp_extraccion,marca,es_demo,es_km0,año_matriculacion,combustible,kilometraje,ubicacion,precio_contado,...,latitud,longitud,garantia,transmision,potencia_cv,puertas,asientos,tipo_carroceria,mes_matriculacion,modelo
0,https://www.autocasion.com/coches-segunda-mano...,2025-06-20 21:10:20.086439,ABARTH,0,0,2022,Gasolina,23213.0,Madrid,22990.0,...,40.4168,-3.7038,12,Manual,179.0,2.0,NaN,Descapotable,12.0,500 C 695C 1.4 16v T-Jet 132kW (180 CV)
1,https://www.autocasion.com/coches-segunda-mano...,2025-06-20 21:10:20.086694,ABARTH,0,0,2018,Gasolina,97814.0,Barcelona,23990.0,...,41.3851,2.1734,12,Automático,170.0,2.0,2.0,Descapotable,12.0,124 SPIDER Spider Turbo Multiair 125kW Auto
2,https://www.autocasion.com/coches-segunda-mano...,2025-06-20 21:10:20.086996,ABARTH,0,0,2022,Gasolina,33602.0,Malaga,22500.0,...,36.7213,-4.4214,12,Manual,165.0,3.0,4.0,Berlina,3.0,595 1.4T JET 121KW


In [16]:
print(df.columns.tolist())

['id_extraccion', 'timestamp_extraccion', 'marca', 'es_demo', 'es_km0', 'año_matriculacion', 'combustible', 'kilometraje', 'ubicacion', 'precio_contado', 'financiacion_disponible', 'latitud', 'longitud', 'garantia', 'transmision', 'potencia_cv', 'puertas', 'asientos', 'tipo_carroceria', 'mes_matriculacion', 'modelo']


In [ ]:
df["garantia"]

0        12
1        12
2        12
3        12
4        12
         ..
88419    36
88420    12
88421    96
88422    84
88423    12
Name: garantia, Length: 88424, dtype: int64

: 

In [18]:
print(df['precio_contado'].unique())

[22990. 23990. 22500. ...  9320. 50650. 26838.]


In [ ]:
SELECT 
            precio_contado,
            año_matriculacion,
            marca,
            kilometraje,
            latitud,
            longitud,
            t.transmision AS transmision_id
        FROM 
            vehiculos
        WHERE
            precio_contado IS NOT NULL
            AND año_matriculacion IS NOT NULL
            AND marca IS NOT NULL
            AND kilometraje IS NOT NULL
            left JOIN transmision t ON vehiculos.transmision_id = t.id

In [ ]:

#usuario tiene que ingresar la informacion del coche para que le modelo pueda predecir el precio del coche
#modelo.predict
#guardar toda la informacion del usuario 
#tengo que saber con que columnas se ha entrenado, y hacer el input de todas esas columnas 

import streamlit as st


#coche_antiguedad = st.number_input("Antigüedad del coche (en años)", min_value=0, max_value=50, value=5)
#coche_kilometraje = st.number_input("Kilometraje del coche (en km)", min_value=0, max_value=500000, value=100000)

#X_usuario = [[coche_antiguedad, coche_kilometraje]] 
#y_usuario_ml = modelo_ml.predict(X_usuario)
#y_usuario_dl = modelo_dl.predict(X_usuario)

#st.write(f"El precio estimado del coche es: {y_usuario_ml[0]:.2f} € (Machine Learning)")
#st.write(f"El precio estimado del coche es: {y_usuario_dl[0]:.2f} € (Deep Learning)")

In [ ]:
import streamlit as st
import pandas as pd
import mysql.connector
from dotenv import load_dotenv
import os
from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from utils.database import fetch_data


df = fetch_data("""
        SELECT v.*, t.transmision AS transmision
        FROM vehiculos v
        LEFT JOIN transmisiones t ON v.transmision_id = t.id;
        """)
#eliminar outliers de precio_contado
df = df[df['precio_contado'] < 1000000]  # Ajusta
with st.expander("🔍 COMPARADOR DE VEHÍCULOS", expanded=True):
    st.header("Comparador de Vehículos")
    st.markdown("Selecciona dos vehículos para comparar sus características principales.")
    
    # Limpieza y conversión de tipos de datos
    df_clean = df.copy()
    
    # Convertir columnas numéricas (asegurando que sean numéricas)
    numeric_cols = {
        'precio_contado': float,
        'kilometraje': float,
        'potencia_cv': float,
        'puertas': int,
        'asientos': int,
        'año_matriculacion': int
    }
    
    for col, dtype in numeric_cols.items():
        if col in df_clean.columns:
            # Convertir a string primero para manejar casos especiales
            df_clean[col] = pd.to_numeric(df_clean[col].astype(str).str.replace(',', '.'), errors='coerce')
            # Rellenar NaN con la mediana
            df_clean[col] = df_clean[col].fillna(df_clean[col].median()).astype(dtype)
    
    # Calcular antigüedad
    df_clean['antiguedad'] = 2025 - df_clean['año_matriculacion']
    
    # Obtener lista única de vehículos (marca + modelo)
    df_clean['marca_modelo'] = df_clean['marca'] + ' ' + df_clean['modelo']
    vehiculos_unicos = df_clean['marca_modelo'].unique()
    
    # Selectores para elegir los vehículos a comparar
    col1, col2 = st.columns(2)
    with col1:
        vehiculo1 = st.selectbox(
            "Selecciona el primer vehículo:",
            options=vehiculos_unicos,
            index=0,
            key="vehiculo1"
        )
    with col2:
        vehiculo2 = st.selectbox(
            "Selecciona el segundo vehículo:",
            options=vehiculos_unicos,
            index=1 if len(vehiculos_unicos) > 1 else 0,
            key="vehiculo2"
        )
    
    # Filtrar datos para los vehículos seleccionados
    try:
        datos_v1 = df_clean[df_clean['marca_modelo'] == vehiculo1].iloc[0]
        datos_v2 = df_clean[df_clean['marca_modelo'] == vehiculo2].iloc[0]
    except IndexError:
        st.error("No se encontraron datos completos para los vehículos seleccionados.")
        st.stop()
    
    # Seleccionar características a comparar
    features = [
        ('precio_contado', 'Precio (€)', 'currency'),
        ('kilometraje', 'Kilometraje (km)', 'number'),
        ('antiguedad', 'Antigüedad (años)', 'number'),
        ('potencia_cv', 'Potencia (CV)', 'number'),
        ('puertas', 'Número de puertas', 'number'),
        ('asientos', 'Número de asientos', 'number'),
        ('transmision', 'Transmisión', 'text')
    ]
    
    # Mostrar información básica
    st.subheader(f"Comparando: {vehiculo1} vs {vehiculo2}")
    
    # Gráfico de radar - solo con características numéricas
    fig_comparacion = go.Figure()
    
    # Obtener máximos para normalización (excluyendo transmision)
    numeric_features = [f for f in features if f[2] != 'text']
    max_values = {col: df_clean[col].max() for col, _, _ in numeric_features if col in df_clean.columns}
    
    for vehiculo, datos, color in [(vehiculo1, datos_v1, '#1f77b4'), (vehiculo2, datos_v2, '#ff7f0e')]:
        valores = []
        categorias = []
        hover_text = []
        
        for col, nombre, tipo in numeric_features:
            if col in datos and pd.notna(datos[col]):
                valor = float(datos[col])  # Asegurar que es float
                
                if col == 'kilometraje':
                    valor_norm = (1 - (valor / max_values[col])) * 100  # Invertir para kilometraje
                else:
                    valor_norm = (valor / max_values[col]) * 100
                
                if tipo == 'currency':
                    display_val = f"{valor:,.0f} €"
                else:
                    display_val = f"{valor:,.0f}"
                
                valores.append(valor_norm)
                categorias.append(nombre)
                hover_text.append(f"{nombre}: {display_val}")
        
        fig_comparacion.add_trace(go.Scatterpolar(
            r=valores,
            theta=categorias,
            fill='toself',
            name=vehiculo,
            line=dict(color=color),
            hoverinfo='text',
            hovertext=hover_text
        ))
    
    # Configurar el gráfico
    fig_comparacion.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 100])
        ),
        margin=dict(t=80, b=20, l=20, r=20),
        title=f"{vehiculo1}<br>{vehiculo2}",
        title_x=0.5,
        height=500,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.1,
            xanchor="center",
            x=0.5
        )
    )
    
    st.plotly_chart(fig_comparacion, use_container_width=True)
    
    # Mostrar tabla comparativa detallada
    st.subheader("Comparación Detallada")
    
    # Crear tabla de comparación
    comparacion_data = []
    for col, nombre, tipo in features:
        if col in datos_v1 and col in datos_v2 and pd.notna(datos_v1[col]) and pd.notna(datos_v2[col]):
            try:
                if tipo == 'currency':
                    valor1 = float(datos_v1[col])
                    valor2 = float(datos_v2[col])
                    display1 = f"{valor1:,.0f} €"
                    display2 = f"{valor2:,.0f} €"
                    diff = ((valor1 - valor2)/valor2*100) if valor2 != 0 else 0
                elif tipo == 'number':
                    valor1 = float(datos_v1[col])
                    valor2 = float(datos_v2[col])
                    display1 = f"{valor1:,.0f}"
                    display2 = f"{valor2:,.0f}"
                    diff = ((valor1 - valor2)/valor2*100) if valor2 != 0 else 0
                else:  # texto
                    display1 = str(datos_v1[col])
                    display2 = str(datos_v2[col])
                    diff = "-"
                
                comparacion_data.append({
                    "Característica": nombre,
                    vehiculo1: display1,
                    vehiculo2: display2,
                    "Diferencia": f"{diff:+.1f}%" if diff != "-" else diff
                })
            except (ValueError, TypeError):
                continue
    
    # Mostrar tabla
    if comparacion_data:
        st.dataframe(
            pd.DataFrame(comparacion_data),
            hide_index=True,
            column_config={
                "Característica": st.column_config.TextColumn(width="medium"),
                vehiculo1: st.column_config.TextColumn(width="small"),
                vehiculo2: st.column_config.TextColumn(width="small"),
                "Diferencia": st.column_config.TextColumn(
                    "Diferencia %",
                    help="Porcentaje de diferencia (Vehículo 1 respecto a Vehículo 2)"
                )
            }
        )
    else:
        st.warning("No hay datos suficientes para mostrar la comparación.")